In [1]:
import numpy as np
import pandas as pd
import mysql.connector
import re

In [2]:
db = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "J@vad5656",
)

In [3]:
mycurser = db.cursor()

In [4]:
mycurser.execute("USE sql_invoicing;")
mycurser.execute("SELECT * FROM clients")

In [5]:
y = []
for x in mycurser:
    y.append(x)

pf = pd.DataFrame(y,columns= mycurser.column_names)
print(pf.head(10))

   client_id         name                   address           city state  \
0          1        Vinte          3 Nevada Parkway       Syracuse    NY   
1          2      Myworks    34267 Glendale Parkway     Huntington    WV   
2          3        Yadel       096 Pawling Parkway  San Francisco    CA   
3          4       Kwideo  81674 Westerfield Circle           Waco    TX   
4          5  Topiclounge          0863 Farmco Road       Portland    OR   

          phone  
0  315-252-7305  
1  304-659-1170  
2  415-144-6037  
3  254-750-0784  
4  971-888-9129  


In [6]:
mycurser.execute("""SELECT 
    i.invoice_id,
    i.number,
    c.name,
    i.invoice_total,
    i.payment_total,
    i.due_date,
    i.payment_date
FROM invoices i
JOIN clients c
USING (client_id)
WHERE i.payment_date IS NOT NULL
ORDER BY invoice_id;""")

y = []
for x in mycurser:
    y.append(x)

table_df = pd.DataFrame(y, columns = mycurser.column_names)
print(table_df.head())

   invoice_id       number         name invoice_total payment_total  \
0           2  03-898-6735  Topiclounge        175.32          8.18   
1           6  75-587-6626        Vinte        157.78         74.55   
2          11  20-848-0181        Yadel        126.15          0.03   
3          13  41-666-1035  Topiclounge        135.01         87.44   
4          15  55-105-9605        Yadel        167.29         80.31   

     due_date payment_date  
0  2019-07-01   2019-02-12  
1  2019-02-18   2019-01-03  
2  2019-01-27   2019-01-11  
3  2019-07-15   2019-01-26  
4  2019-12-15   2019-01-15  


In [7]:
SQL_queries = ["DROP DATABASE IF EXISTS `IMDB_Movies`;"
        , "CREATE DATABASE `IMDB_Movies`;"
        , "USE `IMDB_Movies`;"
        , """CREATE TABLE Top250_Movies(
        movie_id int(4) NOT NULL AUTO_INCREMENT
        , Movie_Name varchar(100) NOT NULL
        , Year int(4) NOT NULL
        , IMDB_Rating float(4) NOT NULL
        , PRIMARY KEY (movie_id)
        ) ENGINE=InnoDB AUTO_INCREMENT=11 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;"""
        , "ALTER TABLE Top250_Movies AUTO_INCREMENT = 1;"]

for query in SQL_queries:
    mycurser.execute(query)
    

In [8]:
def load_csv(addr):
    return pd.read_csv(addr)

In [9]:
addr = "C:\\Users\\moham\\3D Objects\My Files\\Code\Python\\Test Codes\\Web Scraping\\IMDB top 1000\\Top250_Movies_Info_cleaned.csv"
top250_df = load_csv(addr)
top250_df.tail(3)

,Title,Year,Type,Time,Genere,plot,Rating,Voting,Directors,Writers,Stars,User reviews,Critic reviews,Metascore,URL,minutes
247,The Passion of Joan of Arc,1928.0,A,1h 50min,Biography,"In 1431, Jeanne d'Arc is placed on trial on ch...",8.1,50000.0,['Carl Theodor Dreyer'],"['Joseph Delteil', 'Carl Theodor Dreyer']","['Maria Falconetti', 'Eugene Silvain', 'André ...",210.000000,141.000000,82.22549,https://www.imdb.com//title/tt0019254/,110.0
248,Sunrise: A Song of Two Humans,1927.0,A,1h 34min,Drama,A sophisticated urban woman seduces a farmer i...,8.1,48000.0,['F.W. Murnau'],"['Carl Mayer', 'Hermann Sudermann', 'Katherine...","[""George O'Brien"", 'Janet Gaynor', 'Margaret L...",1222.021368,240.029915,82.22549,https://www.imdb.com//title/tt0018455/,94.0
249,The General,1926.0,U,1h 7min,Action,When Union spies steal an engineer's beloved l...,8.1,85000.0,"['Clyde Bruckman', 'Buster Keaton']","['Buster Keaton', 'Clyde Bruckman', 'Al Boasbe...","['Buster Keaton', 'Marion Mack', 'Glen Cavender']",311.000000,119.000000,82.22549,https://www.imdb.com//title/tt0017925/,67.0


In [10]:
for ind, movie_Info in top250_df.iterrows():
    #print(movie_Info)
    title = movie_Info['Title']
    year = int(movie_Info['Year'])
    rating = movie_Info['Rating']
    title = title.replace('\'', '')
    query = f"INSERT INTO Top250_Movies (Movie_Name, Year, IMDB_Rating) VALUES ('{title}', '{year}', '{rating}')"
    #print(query)
    mycurser.execute(query)

# Make sure data is committed to the database
db.commit()

In [12]:
mycurser.execute("""SELECT *
FROM Top250_movies""")

y = []
for ind, x in enumerate(mycurser):
    if ind < 5:
        print(x)
    
    x_dict = {'movie_id' : x[0], 'Movie_Name': x[1], 'Year': x[2], 'IMDB_Rating' :x[3]}
    y.append(x_dict)

table_df = pd.DataFrame(y)
print(table_df.head(3))

(1, 'The Shawshank Redemption', 1994, 9.3)
(2, 'The Godfather', 1972, 9.2)
(3, 'The Dark Knight', 2008, 9.0)
(4, 'The Godfather: Part II', 1974, 9.0)
(5, '12 Angry Men', 1957, 9.0)
   movie_id                Movie_Name  Year  IMDB_Rating
0         1  The Shawshank Redemption  1994          9.3
1         2             The Godfather  1972          9.2
2         3           The Dark Knight  2008          9.0
